### Etape nous permettant d'utiliser Google Colab (Optionnel)

In [ ]:
!git clone https://github.com/joconte/fruit-classifier.git

In [ ]:
%cd fruit-classifier/

### Import des dépendances de FASTAI

In [ ]:
import pkg_resources
pkg_resources.require("fastai==1.0.61")
from fastai.vision import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
torch.cuda.get_device_name(0)

### Chargement de notre dataset

In [ ]:
classes = ['apricot_ok', 'apricot_ripe', 'apricot_unripe', 'banana_ok', 'banana_ripe', 'banana_unripe', 'strawberry_ok', 'strawberry_ripe', 'strawberry_unripe', 'tomato_ok', 'tomato_ripe', 'tomato_unripe']

In [ ]:
for c in classes:
    verify_images("train/"+c, delete=True)
    verify_images("test/"+c, delete=True)

In [ ]:
data = ImageDataBunch.from_folder(path="", train="train", test="test", valid_pct=0.25, ds_tfms=get_transforms(), size=224, bs=32).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3)

### Chargement du model

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)

### Recherche du taux d'apprentisage optimal

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
lr = learn.recorder.min_grad_lr

### Entraînement

In [ ]:
# On lance 4 epochs
learn.fit_one_cycle(4, slice(lr))

In [ ]:
# Puis on lance les epochs un à un tant que le train_loss, le valid_loss baissent et l'accuracy augmente.
learn.fit_one_cycle(1, slice(lr))

# Au bout d'une dizaine d'epoch on obtient une accuracy de 93%

In [ ]:
# On regarde la matrice de confusion
preds, y, losses = learn.get_preds(with_loss=True)

interp = ClassificationInterpretation(learn, preds, y, losses)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
# On décide de débloquer les poids prédéfinis sur le model resnet34
learn.unfreeze()

### Recalcul du taux d'apprentisage optimal

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
lr = learn.recorder.min_grad_lr

### Entrainement avec poids 'unfreeze'

In [ ]:
# On lance les epochs un à un tant que le train_loss, le valid_loss baissent et l'accuracy augmente.
learn.fit_one_cycle(1, slice(lr))

# On obtient ici une accuracy de 93,8%

### Sauvegarde de notre modèle

In [ ]:
learn.export()

### Chargement de notre modèle

In [ ]:
learn = load_learner('')

### Prédiction

In [ ]:
import blah
filename = blah.gui_fname() # <-- Demande de choisir une image
img = open_image(filename)
pred_class, pred_idx, losses = learn.predict(img)
print(pred_class)
print({"prediction": str(pred_class), "scores": sorted(zip(learn.data.classes, map(float, losses)), key=lambda p: p[1], reverse=True)})